# Geoprocessing in Python

## Dataset (1): Mobility Data

In [ ]:
import pandas as pd

#Load Data frames for the Spadina Chinatown Neighbourhood
Spadina20 = pd.read_csv("SpadinaChtn20.csv")
Spadina21 = pd.read_csv("SpadinaChtn21.csv")

#Load Data frames for the Spadina Chinatown Neighbourhood
Liberty20 = pd.read_csv("LibVill20.csv")
Liberty21 = pd.read_csv("LibVill21.csv")

#Add a Year column to the csv file to indicate the year of the data for both neighbourhoods

Spadina20 = Spadina20.assign(Year = 2020)
Spadina21 = Spadina21.assign(Year = 2021)

Liberty20 = Liberty20.assign(Year = 2020)
Liberty21 = Liberty21.assign(Year = 2021)

### Create Spadina 2020 Flow

In [ ]:
!pip install geopandas

import geopandas as gpd
from shapely.geometry import LineString

# Read in dataframe
Spadina20 = Spadina20

# Define the centre of the Spadina BIA (approximately)
Spadina_longlat = (-79.39810845767018, 43.652932760606014)


# Group by evening location lat and long, summing visits and visitors for different dates/times
Spadina20_groupby = Spadina20.groupby(["CEL_LATITUDE","CEL_LONGITUDE"]).agg({
    "Unique_Visitors":'sum', 
    "Daily_Visits": 'sum',
}).reset_index()

# Create straight lines from each location to Spadina Chinatown
lines = [LineString([(row.CEL_LONGITUDE, row.CEL_LATITUDE), Spadina_longlat]) for i, row in Spadina20_groupby.iterrows()]

# Save as a geojson
gdf = gpd.GeoDataFrame(Spadina20_groupby, geometry=lines)
gdf.to_file("Spadinaflows20.geojson", driver="GeoJSON")

### Create Spadina 2021 flow

In [ ]:
#Create Spadina 2021 Flows

# Read in dataframe
Spadina21 = Spadina21

# Define centre of the Spadina BIA (approximately)
Spadina_longlat = (-79.39810845767018, 43.652932760606014)


# Group by evening location lat and long, summing visits and visitors for different dates/times
Spadina21_groupby = Spadina21.groupby(["CEL_LATITUDE","CEL_LONGITUDE"]).agg({
    "Unique_Visitors":'sum', 
    "Daily_Visits": 'sum',
}).reset_index()

# Create straight lines from each location to Spadina Chinatown
lines = [LineString([(row.CEL_LONGITUDE, row.CEL_LATITUDE), Spadina_longlat]) for i, row in Spadina21_groupby.iterrows()]

# Save as a geojson
gdf = gpd.GeoDataFrame(Spadina21_groupby, geometry=lines)
gdf.to_file("Spadinaflows21.geojson", driver="GeoJSON")

### Create Liberty Village 2020 Flow

In [ ]:
#Create Liberty 2020 Flows

# Read in dataframe
Liberty20 = Liberty20

# Define centre of the Liberty Village BIA (approximately)
Liberty_longlat = (-79.41957314863295, 43.63851984317983)


# Group by evening location lat and long, summing visits and visitors for different dates/times
Liberty20_groupby = Liberty20.groupby(["CEL_LATITUDE","CEL_LONGITUDE"]).agg({
    "Unique_Visitors":'sum', 
    "Daily_Visits": 'sum',
}).reset_index()

# Create straight lines from each location to Liberty Village
lines = [LineString([(row.CEL_LONGITUDE, row.CEL_LATITUDE), Liberty_longlat]) for i, row in Liberty20_groupby.iterrows()]

# Save as a geojson
gdf = gpd.GeoDataFrame(Liberty20_groupby, geometry=lines)
gdf.to_file("Libertyflows20.geojson", driver="GeoJSON")

### Create Liberty Village 2021 Flow

In [ ]:
#Create Liberty 2021 Flows

# Read in dataframe
Liberty21 = Liberty21

# Define centre of the Liberty Village BIA (approximately)
Liberty_longlat = (-79.41957314863295, 43.63851984317983)


# Group by evening location lat and long, summing visits and visitors for different dates/times
Liberty21_groupby = Liberty21.groupby(["CEL_LATITUDE","CEL_LONGITUDE"]).agg({
    "Unique_Visitors":'sum', 
    "Daily_Visits": 'sum',
}).reset_index()

# Create straight lines from each location to Liberty Village
lines = [LineString([(row.CEL_LONGITUDE, row.CEL_LATITUDE), Liberty_longlat]) for i, row in Liberty21_groupby.iterrows()]

# Save as a geojson
gdf = gpd.GeoDataFrame(Liberty21_groupby, geometry=lines)
gdf.to_file("Libertyflows21.geojson", driver="GeoJSON")

## Dataset (2): Canadian Historical Business Data

### Select and filter the required data

In [ ]:
# load 2020 Dataset

import pandas as pd
bus2020 = pd.read_csv(r"Canada DB 2020.csv")
bus2020.head(5)

In [ ]:
#Filtering data to include only Toronto businesses by postcal code
bus2020_TO = bus2020.loc[bus2020["ZIPP4F"].str.startswith("M", na = False)].copy().reset_index()

#Filtering data to include only the businesses in the neighbourhoods of interest
bus2020_SLV = bus2020_TO[(bus2020_TO["ZIP"] == "M5T") | (bus2020_TO["ZIP"] == "M5V") | (bus2020_TO["ZIP"] == "M6K")].copy().reset_index()

#Create a new column with the full postal code by combining the first and second half of the codes which was broken down into two columns originally
bus2020_SLV["Postal_Code"] = bus2020_SLV["ZIP"] + bus2020_SLV["ZIP4"]

In [ ]:
#To extract the the 3-digit NAICS code from the original 6-digit ones
bus2020_SLV["NAICS3"] = bus2020_SLV["NAICS"].astype("str").str[:3]

#To filter only full service restuarants and drinking places based on the the first three digits of NAICS code
rest2020_SLV = bus2020_SLV[bus2020_SLV["NAICS3"] == "722"].reset_index(drop=True).copy()


### Prepare the Address Information for Geomapping

In [ ]:
#Combine the individual pieces of address information into one full address
full_address = rest2020_SLV.apply(lambda x: f'{x["STADDR"]} {x["STCITY"]} {x["STATE"]} ', axis = 1)

#Seperate the full data frame into four portion before calling the geocoding as it may trigger a time-out in one go
fa1= full_address[0::4]
fa2= full_address[1::4]
fa3= full_address[2::4]
fa4= full_address[3::4]

### Use the Geocoder to Find Coordinates for All Addresses

In [ ]:
#Import the geopy package
!pip install geopy

#Import geocoders from the geopy package
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="jill")

In [ ]:
location = fa1.apply(lambda x: geolocator.geocode(x, timeout=None))

In [ ]:
location2 = fa2.apply(lambda x: geolocator.geocode(x, timeout=None))

In [ ]:
location3 = fa3.apply(lambda x: geolocator.geocode(x, timeout=None))

In [ ]:
location4 = fa4.apply(lambda x: geolocator.geocode(x, timeout=None))

In [ ]:
#Combining all segments of data into one
full_address = pd.concat([location,location2,location3,location4]).sort_index()

#Insert the mapped coordinates into the original data frame, code as nan if not the address was not found
import numpy as np
rest2020_SLV["lat"] = full_address.apply(lambda x: x.latitude if x else np.nan)
rest2020_SLV["long"] = full_address.apply(lambda x: x.longitude if x else np.nan)

In [ ]:
#Select and rearrage the final data frame for export to QGIS
rest2020_SLV_QGIS = rest2020_SLV[["CONAME",
                                "STADDR",
                                "STCITY",
                                "STATE",
                                "ZIPP4F",
                                "NAICS",
                                "NAICSD",
                                "NAICS3",
                                "Postal_Code",
                                "lat",
                                "long"]]
rest2020_SLV_QGIS

rest2020_SLV_QGIS.to_csv("latlongSLV.csv", index = False, header = True)